# Imports

In [2]:
from torchvision import datasets
import torch
import torch.nn as nn
from torchvision.transforms import ToTensor
from tqdm import tqdm

# Prepare dataset

In [3]:
batch_size = 64

In [4]:
dataset1 = datasets.FashionMNIST("./data", train = True, download = True, transform = ToTensor())
dataset2 = datasets.FashionMNIST("./data", train = False, download = True, transform = ToTensor())

In [5]:
train_loader = torch.utils.data.DataLoader(dataset1, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size = batch_size, shuffle = True)

In [7]:
for X, y in train_loader:
    print(X.shape)
    print(y.shape)
    break

torch.Size([64, 1, 28, 28])
torch.Size([64])


# Define model

In [36]:
class Model(nn.Module):

    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, X):
        return self.layers(X)

class ConvModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 1, 3, padding=1),
            nn.ReLU()
        )
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(28*28, 10)
    
    def forward(self, X):
        out = self.layers(X)
        return self.linear(self.flatten(out))

# Train loop

In [10]:
def train(dataloader, model, loss_fn, optimizer, device):
    model.train()
    test_loss = 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)

        loss = loss_fn(pred, y)

        loss.backward()

        optimizer.step()
        optimizer.zero_grad()
        
        test_loss += loss.item()
    
    return test_loss / len(dataloader)



def test(dataloader, model, loss_fn, device):
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for batch, (X, y) in enumerate(dataloader):
            X, y = X.to(device), y.to(device)

            pred = model(X)
            
            test_loss += loss_fn(pred, y).item()

            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= len(dataloader) # divide by number of batches
    correct /= len(dataloader.dataset) # divice by number of samples

    return test_loss, correct


In [42]:
EPOCHS = 5
device = torch.device("cuda")
model = ConvModel().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)

losses = []
accuracy = []

for _ in tqdm(range(EPOCHS), unit = "epochs"):
    train(train_loader, model, loss_fn, optimizer, device)
    test_loss, correct = test(test_loader, model, loss_fn, device)

    losses.append(test_loss)
    accuracy.append(correct)

100%|██████████| 5/5 [00:30<00:00,  6.04s/epochs]


In [43]:
losses, accuracy

([0.5267180461602606,
  0.4906780950392887,
  0.4607845528680048,
  0.4840457943404556,
  0.4305367640628936],
 [0.8066, 0.8193, 0.8377, 0.8272, 0.8494])

# Save model

In [44]:
torch.save(model.state_dict(), "model.pth")

# Load model

In [47]:
model = ConvModel().to(device)
test(test_loader, model, loss_fn, device)

(2.3027840769214993, 0.1)

In [48]:
model.load_state_dict(torch.load("model.pth", weights_only = True))
test(test_loader, model, loss_fn, device)

(0.4317399589878738, 0.8494)